In [33]:
    
from mesa.space import MultiGrid
from mesa import Model
from mesa.time import RandomActivation
import numpy as np
from queue import Queue


class ElFarolBar(Model):
    def __init__(self, crowdthreshold=60,num_strategies=10,memory_size = 10,width = 100,height = 100,N=100):
        self.running = True 
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, True)
        self.history = Queue()
        for n in np.random.randint(0,100,size = memory_size*2):
            self.history.put(n)
        self.attendance = self.history[0]


        for i in range(self.num_agents):
            a = BarCustomer(i,self, num_strategies,memory_size,crowdthreshold) 
            self.schedule.add(a)
        self.datacollector = DataCollector(
            model_reporters ={'Customers':"attendance"})
            #agent_reporters={"Bar": "Bar"})
    def step(self):
        self.attendance = 0
        self.datacollector.collect(self)
        self.schedule.step()
        self.history.get()
        self.history.put(self.attendance)


In [36]:
from mesa import Agent
from mesa.datacollection import DataCollector

class BarCustomer(Agent):
    def __init__(self, unique_id, model,num_strategies,memory_size,crowdthreshold):
        super().__init__(unique_id, model)
        self.strategies = np.random.rand(num_strategies+1)-1
        self.best_strategy = self.strategies[0]
        self.attend = False
        self.memory = np.zeros(memory_size)
        self.crowdthreshold = crowdthreshold
        self.update_strategies()
#        print(self.model.history)

    #need to add step method here, to activate make_decision
    def step(self):
        prediction = self.predict_attendance()
        if prediction<=self.crowdthreshold:
            self.attend = True
            self.model.attendance = self.model.attendance +1
        else: 
            self.attend = False
        self.update_strategies()
    
    def update_strategies(self):
        best_score = self.memory_size*100+1
        score = 0 
    
    def predict_attendance(self):
        pass
    

In [30]:
model = ElFarolBar(N=5)

In [17]:
model.history

array([83, 98,  0, 24, 51, 87, 23, 50, 66, 37, 58, 41, 16,  3,  9,  3, 17,
       41, 40, 13])

In [18]:
model.step()

one step
one step
one step
one step
one step
